# Wichtige Funktionen zur Auswertung von Versuchen. 
Diese Vorlage kann kopiert werden und dann für den jeweiligen Versuch genutzt werden. 

Zunächst soll die Versuchsummer des jeweiligen Versuches angegeben werden, damit das automatische Speichern der Grafiken in das richtige Verzeichnis funktioniert.

Dieses Dokument ist an vielen Stellen (stark) an *Luca Hafner* orientiert.

## Versuchsnummer:

In [15]:
versuchsnummer = "000"

## Import aller wichtigen Libaries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.transforms as transforms
%matplotlib inline

import numpy as np
from numpy import exp, sqrt, log, pi

from scipy import odr
import scipy.optimize
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy.stats import poisson
from scipy.integrate import quad
from scipy.signal import find_peaks
from scipy.signal import argrelextrema, argrelmin, argrelmax
from scipy.special import factorial

import os
import os.path

import pandas as pd
import pylab as py
from IPython.display import display, Math, Latex, HTML

import sympy as sp
from sympy import separatevars

### Erstellen der dedizierten Dartei zum Speichern der Werte

Folgender Code sollte einmal ausgeführt werden, damit direkt die Datei zum Speichern erstellt werden kann. Unbedingt daran denken, die Versuchsnummer anzupassen, die Werte anderer Versuche könnten ansonsten verloren gehen. Es gibt jedoch eine sicherheits Kopie. 

In [63]:
def create_tex_result_values(filePath = "Versuche/" + versuchsnummer + "/Auswertung", fileName = "python-results.sty"):
    """
    Diese Method erstellt automatisch eine tex-Datei, in dem Messwerte bzw. deren Ergebnisse, Tabellen weiteres als variable gespeichert werden, die hier in diesem Python-code bestimmt werden.
    Die File wird unter *Versuche/${versuchsnummer}$/Auswertung/python-results.tex* zufinden sein. Am einfachsten ist es jedoch die Parameter frei zuhalten, da ansonsten auch das Verzeichnis in der *main.tex* 
    angepasst werden muss. 

    Parameter
    ----------
    **filePath**: 
        relatives Verzeichnis, falls der Speicherort explizit geändert werden soll.

    **fileName**: 
        neuer Name, falls die Datei besonders heißen soll.
    """
    # Erstellung der file

    path = filePath + "/" + fileName

    if os.path.isfile(path):
        pass
    else:
        with open(path, 'w') as file:
            file.write("% Dies ist eine automatisch generierte Datei. Hier werden automatisiert Variablen fuer Formeln, Ergebnisse und Tabellen erstellt. \n% Bitte nicht in diese Datei schreiben. Informationen koennten geloescht oder nicht richtig verarbeitet werden. \n\n%  _          _   _       _______      \n% | |        | | ( )     |__   __|\n% | |     ___| |_|/ ___     | | _____  __\n% | |    / _ \ __| / __|    | |/ _ \ \/ /\n% | |___|  __/ |_  \__ \    | |  __/>  < \n% |______\___|\__| |___/    |_|\___/_/\_\ \n\n\n") 

        with open(filePath + "/python-results_BackUp.sty", 'w') as bFile:
            bFile.write("% Dies ist eine automatisch generierte Datei. Hier wird dediziert ein Back-Up erstellt, damit Werte nicht verloren gehen. \n\n%  _          _   _       _______      \n% | |        | | ( )     |__   __|\n% | |     ___| |_|/ ___     | | _____  __\n% | |    / _ \ __| / __|    | |/ _ \ \/ /\n% | |___|  __/ |_  \__ \    | |  __/>  < \n% |______\___|\__| |___/    |_|\___/_/\_\ \n\n\n") 

    # print(filePath + "/" + fileName)
    # print(versuchsnummer)

    return path

create_tex_result_values()
pyPath = create_tex_result_values()



<>:23: SyntaxWarning: invalid escape sequence '\ '
<>:26: SyntaxWarning: invalid escape sequence '\ '
<>:23: SyntaxWarning: invalid escape sequence '\ '
<>:26: SyntaxWarning: invalid escape sequence '\ '
C:\Users\finnz\AppData\Local\Temp\ipykernel_9780\3535738337.py:23: SyntaxWarning: invalid escape sequence '\ '
  file.write("% Dies ist eine automatisch generierte Datei. Hier werden automatisiert Variablen fuer Formeln, Ergebnisse und Tabellen erstellt. \n% Bitte nicht in diese Datei schreiben. Informationen koennten geloescht oder nicht richtig verarbeitet werden. \n\n%  _          _   _       _______      \n% | |        | | ( )     |__   __|\n% | |     ___| |_|/ ___     | | _____  __\n% | |    / _ \ __| / __|    | |/ _ \ \/ /\n% | |___|  __/ |_  \__ \    | |  __/>  < \n% |______\___|\__| |___/    |_|\___/_/\_\ \n\n\n")
C:\Users\finnz\AppData\Local\Temp\ipykernel_9780\3535738337.py:26: SyntaxWarning: invalid escape sequence '\ '
  bFile.write("% Dies ist eine automatisch generierte D

## Definition der Funktionen

---
## Table of Contents:
* [Import von Dateien](#Import-von-Dateien)
* [Fehlerrechnung einer gegebenen Formel](#Fehlerrechnun-einer-gegebenen-Formel)
* [Automatisierter Latex-Export](#Automatisierter-Latex-Export)
    * [Export der Latex Formel](#Export-der-Latex-Formel)
    * [Ergebnisse plus Fehler](#Ergebnisse-plus-Fehler)
    * [Export als Tabelle](#Export-als-Tabelle)
---

##  Import von Dateien
Es sollen hier Funktionen definiert werden, die **CSV** und **TXT** -Dateien importieren. Diese sollen dann einfacher automatisiert in Latex-Tabellen eingefügt werden. 

In [ ]:
"""
    Hier sollen Messwerte aus CSV und TXT
"""

def placeholder():
    return

Versuche/000/Auswertung/python-results.sty


## Fehlerrechnung einer gegebenen Formel

**Gauss Fehler Fortpflanzung (GFF)**:

Die *Methode gff* nimmt zwei Argumente, zum einen eine *sympy (sp)* Funktion, zum anderen einen Array aller fehlerbehafteter Größen.

In [19]:
def gff(func, errPronePar):
    """
    Gaussian Error Propergation
    """
    # Variablen definieren
    error = 0
    errProneParamters = []
    for errPar in errPronePar:
        d = sp.symbols('d' + errPar.name)
        partial = sp.diff(func, errPar) * d  # Die Funktion wird nach der fehlerbehafteten Variable abgeleitet
        error = error + partial**2 # Fehler werden quadratisch aufsummiert
        errProneParamters.append((errPar,d))
    absolut_err=sp.simplify(sp.sqrt(error),rational = True)             
    relativ_err=sp.simplify(sp.sqrt(error/func**2),rational = True)
    return absolut_err, relativ_err, errProneParamters

Beispiel gff:

In [20]:
# Drei Messgrößen x, y und z werden definiert.
x = sp.Symbol('x')
y = sp.Symbol('y')
z = sp.Symbol('z')

params = [x, y, z]

# Darstellen einer Funktion
f = x**x

# Unsicherheiten (bspw. Ablesefehler)
dx_val = 0.05  
dy_val = 0.02 
dz_val = 0.10  


# Ausführen von gff
abs_err, rel_err, param_symbols = gff(f, params)

print("Absoluter Fehler (delta f):")
sp.pprint(abs_err)

print("\nRelativer Fehler (delta f / f):")
sp.pprint(rel_err)

Absoluter Fehler (delta f):
   ________________________
  ╱   2  2⋅x             2 
╲╱  dx ⋅x   ⋅(log(x) + 1)  

Relativer Fehler (delta f / f):
   ___________________
  ╱   2             2 
╲╱  dx ⋅(log(x) + 1)  


## Automatisierter Latex-Export

Es wird eine dedizierte Datei für die Ergebnisse geben, in der die Ergebnisse als Latex-Variable stehen. Diese können dann im Latex-Code implementiert werden. Wurde eine Größe falsch bestimmt, so kann diese im Python-Code neu berechnet werden und wird automatisiert im neuen Dokument (nach einem *Recompile*) korrigiert dargestellt.

## Ersstellen einer dedizierten **TEX** -Datei

Diese beinhaltet:
* Formel, sowie Formel nach gff
* Berechnete Werte und deren Fehler
    * Wert + Fehler
    * Tabellen Export

### Export der Latex Formel

Die folgende Funktion transformiert eine Formel in Latex. Diese Formel kann entweder hier im Notebook einfach kopiert werden oder als Variable in der *pyton-results.tex*.

In [57]:
def function_to_latex(f, texVarName, texCom):
    """
    Zeigt die Formel als gerenderte Math-Darstellung und darunter
    den Latex-Quelltext, der per Button kopiert werden kann. (Für leichtere Benutzung als HTML).
    
    Zudem wird die Latexformel als Variable in Latex gespeichert.

    Parameters
    ----------
    **f**: sympy function

    **texVarName**: Einzigartiger Name der Variablen. Dieser wird zum überschreiben alter Formel gebraucht.

    **texCom**: setzt den newcommand-Kürzel für latex fest. Setzte kein Backslash! Auch texCom muss einzigartig gesetzt werden.
    """
    # print("Die gegebene Funktion lautet: \n")
    display(Math(sp.latex(f, long_frac_ratio=2).replace('d__', r'\Delta ')))

    # print("Hier ist der dazugehörige Latex code: \n")
    latex_str = sp.latex(f, long_frac_ratio=2).replace('d__', r'\Delta ')
    html = f"""
    <div style="margin-top:0.5em;">
        <code id="latex-code-{id(f)}">
            {latex_str}
        </code>
        <button onclick="
            const tex_as_txt = document.getElementById('latex-code-{id(f)}').innerText;
            navigator.clipboard.writeText(tex_as_txt)
        " style="
            margin-left:8px;
            padding:2px 6px;
            cursor:pointer;
        ">
            Kopieren
        </button>
    </div>
    """
    display(HTML(html))


    # Hinzufuegen bzw. Ueberschreieben der Formel in die Sammlung
    with open(pyPath, 'r') as file:
        lines = file.readlines()
    found = False

    with open(pyPath, 'r') as file:
        for lineNum,  line in enumerate(lines, 1):
            if texVarName in line: # Checkt, ob der Variablen Name bereits vergeben ist.
                print(f'{texVarName} is at line {lineNum}') 
                lines[lineNum] = "newcommand{\\" + texCom + "}{" + latex_str + "}\n"
                found = True

    if not found:
        print("Die neue Funktion wurde hinzugefügt")
        with open(pyPath, 'w') as file:
            file.writelines(lines) # Schreibt den alten Stand
            file.write("\n" + texVarName + "\n")
            file.write("newcommand{\\" + texCom + "}{" + latex_str + "}\n\n")
    else:
        print("Die alte Funktion wurde erfolgreich überschrieben.")
        with open(pyPath, 'w') as file:
            file.writelines(lines)

Beispiel function_to_latex:

In [58]:
f = y**2

line = function_to_latex(f, "Parabel", "par")

<IPython.core.display.Math object>

Die neue Funktion wurde hinzugefügt


### Ergebnisse plus Fehler

In [23]:
def calc_with_err(func, errFunc, values):
    """
    Methode zum berechnen von Werten und deren Fehler.

    Parameter
    ----------
    **func**: 
        sympy Funktion mit Parametern. 

    **errFunc**: 
        die zu func gehöhrende sympy Fehler-Funktion nach gff

    **values**: 
        Werte, die in die Funktionen eingesetzt werden.
        Als array von Tupeln der Form [(a,da),(b,db),...] oder als array/liste [a,da,b,db,...] 
        (Reihenfolge muss die sein, in der die Argumente in der Funktion genommen werden)
    """

    #Falls der Input in mehrere Tupel aufgeteilt ist, werden diese zu einem Array zusammengefügt 
    if (np.ndim(values) != 1):                    
        values = np.concatenate(values)
    #print(values)
    result = func(*values[::2])
    uncertainty = errFunc(*values)
    return result, uncertainty

In [54]:
def calc_everything(function, texVarName, texCom, params, data, params_without_error=[]):
    """
    Nimmt als imput eine Funktion und die Information welche Parameter fehlerbehaftet sind. Zudem 

    Parameter
    ----------
    **function**: 
        sympy Funktion mit Parametern. In diese werden die Messwerte eingesetzt.

    **texVarName**: 
        Einzigartiger Name der Variablen. Dieser wird zum überschreiben alter Formel gebraucht.

    **texCom**:     
        setzt den newcommand-Kürzel für latex fest. Setzte kein Backslash! Auch texCom muss einzigartig gesetzt werden.

    **params**: 
        Parameter der Funktion. Diese werden als Array von Sympy-Symbolen gebraucht. Bspw. [x, y, z]

    **data**:
        2D-Array mit den Messdaten, sodass die Zeilen die Form haben: [Parameter 1, Fehler Parameter 1, Parameter 2,...]
        Die Funktion wird zeilenweise angewandt. Wird kein Fehler für einen Parameter angenommen, kann diese Spalte entweder mit dem Wert 0 
        an die Funktion gegeben werden oder ganz weggelassen werden. Dann muss allerdings der betreffende Parameter bei params_without_error angegeben werden.

    **params_without_error**: 
        Alle Parameter zu denen kein Fehler explizit in den Daten angegeben ist. Dieser wird auf 0 gesetzt und kommt dann
        auch nicht in der Latex Form der Fehlerformel vor

    """
    # Expand data to include the uncertainty 0 for values with no assigned uncertainty
    # Hat zubeginn die Form: [[0. 0. 0. 0. 0. 0. ...]]. 
    # Bei jeder Iteration werden dann die Parameter und deren Fehler hinzugefügt: i=1 -> [[par1 errPar1 0. 0. 0. 0. ...]]
    exp_data = np.zeros((data.shape[0],data.shape[1]+len(params_without_error)))
    i = 0      # läuft durch die Parameter
    j = 0      # läuft durch die expanded data
    z = 0      # läuft durch die eingegebene data, also die Messwerte und deren Fehler
    # Läuft durch jeden Parameter und seinen Fehler
    while (i < len(params)):
        # Checkt, ob der Parameter Fehlerbehaftet ist. Wenn, dann wird an j-ter Stelle des exp_data der z-te Parameter aus data einegfügt.
        if (params[i] in params_without_error):
            exp_data[:,j] = data[:,z]
            i = i + 1
            j = j + 2
            z = z + 1
        else:
            exp_data[:,j] = data[:,z]
            exp_data[:,j+1] = data[:,z+1]
            i = i + 1
            j = j + 2
            z = z + 2
    # print(exp_data)

    # Create variable that stores parameters that have no assigned uncertainty    
    params_with_error = []
    j = 0
    for n in np.arange(0,len(params)):
        if not (params[n] in params_without_error):
            params_with_error.append(params[n])
            j = j + 1
    
    # Get the given function and error function as numpy functions
    f = sp.lambdify(params,function)
    absolut_err, relativ_err, parameters = gff(function,params) # Gauss Fehlerfortpflanzung
    err_abs = sp.lambdify(np.concatenate(parameters),absolut_err)

    # Calculate the results for each row of data
    results = np.zeros((data.shape[0],2))
    for n in np.arange(0,data.shape[0]):
        results[n,:] = calc_with_err(f, err_abs, exp_data[n,:])
        # print(calc_with_err(f, err_abs, exp_data[n]))
    
    if (len(results) < 10):
        print("Results:")
        print(results)

    # Substitutes 0 for the uncertainty of the parameters without error, so it doesnt show up in the Latex Code
    for p in params_without_error:
        absolut_err = absolut_err.subs('d'+p.name,0)
    for p in params_without_error:
        relativ_err = relativ_err.subs('d'+p.name,0)

    # Wiedergabe der Formeln in Latex
    function = sp.simplify(function,symbols = params, rational= True)
    function = sp.separatevars(function)
    
    print("gegebene Funktion:")
    function_to_latex(function, texVarName, texCom)

    print("Formel des absoluten Fehlers der gegebenen Funktion:")
    function_to_latex(absolut_err, "errAbs" +  texVarName, "errAbs" +  texCom)

    print("Formel des relativen Fehlers der gegebenen Funktion:")
    function_to_latex(relativ_err, "errRel" +  texVarName, "errRel" +  texCom)

    return(results)


Beispiel calc_with_err:

In [25]:
#Beispieleingabe für die Funktion "uncertainty" (Daten aus Jens' Fehlerrechner)
a,b,c,d = sp.symbols('a b c d')
params = a,b,c,d
function = (a*b**2*sp.sqrt(c) - d)
testd = np.array([[0.8,0.3,4.2,0,2.4,0.1,7,0.12], 
                  [6.8,6.3,4.2,6,2.4,6.1,7,6.12]])

z = calc_everything(function, "beispielFunktion", "bspFunc", params, testd, params_without_error = [])

Results:
[[ 14.86221639   8.21184997]
 [178.82883934 606.0902399 ]]
gegebene Funktion:


<IPython.core.display.Math object>

Die neue Funktion wurde hinzugefügt
Formel des absoluten Fehlers der gegebenen Funktion:


<IPython.core.display.Math object>

Die neue Funktion wurde hinzugefügt
Formel des relativen Fehlers der gegebenen Funktion:


<IPython.core.display.Math object>

Die neue Funktion wurde hinzugefügt


## Export der Werte

In [26]:
def placeholder():
    """
    Erklärung der Methode
    """
    return

### Export als Tabelle

In [27]:
def placeholder():
    """
    Erklärung der Methode
    """
    return

# Back Up erstellen

In [28]:
def placeholder():
    """
    Erklärung der Methode
    """
    return